# 04 - Pipeline Final e Geração da Submissão

**🎯 PROPÓSITO DESTE NOTEBOOK:**
Este notebook contém o pipeline final para o Hackathon Forecast Big Data 2025. O processo consiste em:

1. **Carregamento dos Dados Brutos:** Carregar os dados de 2022 (transações, produtos, pdvs)
2. **Engenharia de Features:** Função completa para processar os dados e criar features
3. **Treinamento do Modelo Final:** Treinar o LightGBM com 100% dos dados de 2022
4. **Geração do Grid de Previsão:** Criar o dataframe base para as 5 semanas de Janeiro/2023
5. **Previsão e Submissão:** Gerar previsões e salvar arquivos CSV e Parquet

**🚀 PIPELINE COMPLETO:**
Este notebook executa todo o processo de ponta a ponta, da engenharia de features até a geração da submissão final.

In [13]:
import pandas as pd
import numpy as np
import pickle
import warnings
import lightgbm as lgb
from datetime import datetime, timedelta
import gc
import os

# Use o Polars para a engenharia de features, como no notebook 02
import polars as pl

warnings.filterwarnings('ignore')
print('📚 Bibliotecas carregadas com sucesso!')
print('🎯 Iniciando Pipeline Final para Submissão')

📚 Bibliotecas carregadas com sucesso!
🎯 Iniciando Pipeline Final para Submissão


In [14]:
def engenharia_de_features(df_transacoes, df_produtos, df_pdvs):
    """
    Aplica a engenharia de features completa usando Polars para eficiência.
    Recebe os dataframes brutos e retorna um dataframe pandas com as features.
    
    Esta função implementa EXATAMENTE a mesma lógica do notebook 02-Feature-Engineering-Dask.ipynb
    """
    print("🔧 Iniciando engenharia de features com Polars...")
    
    # Converter para Polars para performance
    transacoes_pl = pl.from_pandas(df_transacoes)
    produtos_pl = pl.from_pandas(df_produtos)
    pdvs_pl = pl.from_pandas(df_pdvs)
    
    print(f"   • Transações: {len(transacoes_pl):,} registros")
    print(f"   • Produtos: {len(produtos_pl):,} registros")
    print(f"   • PDVs: {len(pdvs_pl):,} registros")
    
    # === INÍCIO DA LÓGICA ADAPTADA DO NOTEBOOK 02 ===
    
    # 1. Renomear colunas para consistência (CORREÇÃO BASEADA NOS DADOS REAIS)
    print("📝 Renomeando colunas para consistência...")
    
    # Transações: renomear para nomes padronizados
    transacoes_pl = transacoes_pl.rename({
        'internal_product_id': 'produto_id',
        'internal_store_id': 'pdv_id',
        'reference_date': 'semana',
        'quantity': 'quantidade'
    })
    
    # Produtos: usar as colunas que existem realmente
    produtos_pl = produtos_pl.rename({
        'produto': 'produto_id'  # A coluna chave é 'produto', não 'internal_product_id'
    })
    
    # PDVs: usar as colunas que existem realmente  
    pdvs_pl = pdvs_pl.rename({
        'pdv': 'pdv_id'  # A coluna chave é 'pdv', não 'internal_store_id'
    })
    
    print("✅ Colunas renomeadas com sucesso!")
    
    # 2. Joins dos dados (agora vai funcionar)
    print("🔗 Fazendo joins dos dados...")
    dados = transacoes_pl.join(produtos_pl, on='produto_id', how='left').join(pdvs_pl, on='pdv_id', how='left')
    
    # 3. Conversão de data e ordenação - CORREÇÃO AQUI
    print("📅 Processando datas...")
    # Verificar o tipo da coluna semana antes de converter
    semana_dtype = str(dados["semana"].dtype)
    print(f"   • Tipo atual da coluna semana: {semana_dtype}")
    
    if semana_dtype not in ["Date", "Datetime"]:
        # Se não for Date nem Datetime, converter
        dados = dados.with_columns(pl.col("semana").str.to_datetime("%Y-%m-%d"))
    
    dados = dados.sort(["pdv_id", "produto_id", "semana"])
    
    # 4. Features temporais (mês, semana do ano, sin/cos para sazonalidade)
    print("🕒 Criando features temporais...")
    dados = dados.with_columns([
        pl.col("semana").dt.month().alias("mes"),
        pl.col("semana").dt.week().alias("semana_ano")
    ])
    
    # Features cíclicas para capturar sazonalidade
    dados = dados.with_columns([
        (pl.col("mes") * (2 * np.pi / 12)).sin().alias("mes_sin"),
        (pl.col("mes") * (2 * np.pi / 12)).cos().alias("mes_cos")
    ])
    
    # 5. Features de Lag (valores passados)
    print("⏪ Criando features de lag...")
    lags = [1, 2, 3, 4]
    for lag in lags:
        dados = dados.with_columns(
            pl.col("quantidade").shift(lag).over(["pdv_id", "produto_id"]).alias(f"quantidade_lag_{lag}")
        )
    
    # 6. Features de Rolling Window (médias móveis, etc.)
    print("📊 Criando features de rolling window...")
    dados = dados.with_columns([
        pl.col("quantidade").rolling_mean(window_size=4, min_periods=1).over(["pdv_id", "produto_id"]).alias("quantidade_media_4w"),
        pl.col("quantidade").rolling_max(window_size=4, min_periods=1).over(["pdv_id", "produto_id"]).alias("quantidade_max_4w"),
        pl.col("quantidade").rolling_min(window_size=4, min_periods=1).over(["pdv_id", "produto_id"]).alias("quantidade_min_4w")
    ])
    
    # 7. Features de Hash para reduzir dimensionalidade - CORREÇÃO AQUI
    print("🔢 Criando features de hash...")
    dados = dados.with_columns([
        (pl.col("pdv_id").hash() % 100).cast(pl.Int8).alias("pdv_hash"),
        (pl.col("produto_id").hash() % 1000).cast(pl.Int16).alias("produto_hash"),
        ((pl.col("pdv_id").cast(str) + "_" + pl.col("produto_id").cast(str)).hash() % 10000).cast(pl.Int16).alias("pdv_produto_hash")
    ])
    
    # 8. Features históricas por combinação PDV/Produto - CORREÇÃO AQUI
    print("📈 Criando features históricas...")
    dados = dados.with_columns([
        pl.col("quantidade").mean().over(["pdv_id", "produto_id"]).alias("hist_mean"),
        pl.col("quantidade").std().over(["pdv_id", "produto_id"]).alias("hist_std"),
        pl.col("quantidade").max().over(["pdv_id", "produto_id"]).alias("hist_max"),
        pl.col("quantidade").count().over(["pdv_id", "produto_id"]).cast(pl.Int16).alias("hist_count")  # Int16 ao invés de Int8
    ])
    
    # 9. Preencher NaNs que surgiram dos lags/rolling
    print("🔧 Preenchendo valores missing...")
    dados = dados.fill_null(0)
    
    # === FIM DA LÓGICA ADAPTADA DO NOTEBOOK 02 ===
    
    print("✅ Engenharia de features concluída!")
    
    # Converter de volta para pandas
    df_final = dados.to_pandas()
    print(f"   • Shape final: {df_final.shape}")
    print(f"   • Features criadas: {len(df_final.columns)}")
    
    return df_final

print("🛠️ Função de engenharia de features definida com sucesso!")

🛠️ Função de engenharia de features definida com sucesso!


In [15]:
# Carregar dados brutos de 2022 diretamente
print("📂 Carregando dados brutos de 2022...")

# Carregar os dados brutos dos arquivos parquet originais
df_transacoes_2022 = pd.read_parquet('../data/part-00000-tid-5196563791502273604-c90d3a24-52f2-4955-b4ec-fb143aae74d8-4-1-c000.snappy.parquet')
df_produtos = pd.read_parquet('../data/part-00000-tid-7173294866425216458-eae53fbf-d19e-4130-ba74-78f96b9675f1-4-1-c000.snappy.parquet')
df_pdvs = pd.read_parquet('../data/part-00000-tid-2779033056155408584-f6316110-4c9a-4061-ae48-69b77c7c8c36-4-1-c000.snappy.parquet')

print(f"   • Transações: {df_transacoes_2022.shape}")
print(f"   • Produtos: {df_produtos.shape}") 
print(f"   • PDVs: {df_pdvs.shape}")

# Aplicar engenharia de features usando a função criada
print('\n🔧 Aplicando engenharia de features...')
dados_treino_com_features = engenharia_de_features(df_transacoes_2022, df_produtos, df_pdvs)

print(f'\n📊 Dados com features processados:')
print(f'   • Shape: {dados_treino_com_features.shape}')
print(f'   • Período: {dados_treino_com_features["semana"].min()} até {dados_treino_com_features["semana"].max()}')

# Aplicar otimização de memória (downcasting) como no seu trabalho anterior
print('\n📊 Otimização de memória e tratamento de missing values...')

# Downcasting para otimizar memória
print('🔽 Aplicando downcasting...')
for col in dados_treino_com_features.select_dtypes(include=[np.number]).columns:
    original_dtype = dados_treino_com_features[col].dtype
    if dados_treino_com_features[col].dtype.kind in ['i', 'u']:  # Inteiros
        dados_treino_com_features[col] = pd.to_numeric(dados_treino_com_features[col], downcast='integer')
    else:  # Floats
        dados_treino_com_features[col] = pd.to_numeric(dados_treino_com_features[col], downcast='float')

# Otimizar colunas categóricas
print('📂 Otimizando categóricas...')
for col in dados_treino_com_features.select_dtypes(include=['object']).columns:
    if col not in ['semana']:  # Preservar datetime
        nunique = dados_treino_com_features[col].nunique()
        total_rows = len(dados_treino_com_features)
        if nunique / total_rows < 0.5:  # Se <50% valores únicos, usar category
            dados_treino_com_features[col] = dados_treino_com_features[col].astype('category')

# Tratamento inteligente de missing values para distributor_id
if 'distributor_id' in dados_treino_com_features.columns:
    print('🔧 Tratando missing values em distributor_id...')
    if dados_treino_com_features['distributor_id'].dtype.name == 'category':
        if -1 not in dados_treino_com_features['distributor_id'].cat.categories:
            dados_treino_com_features['distributor_id'] = dados_treino_com_features['distributor_id'].cat.add_categories([-1])
    dados_treino_com_features['distributor_id'] = dados_treino_com_features['distributor_id'].fillna(-1)

print('✅ Otimização concluída!')

# Definir features e target - CORREÇÃO AQUI PARA EXCLUIR DATETIME
target = 'quantidade'
exclude_features = [
    'pdv_id', 'produto_id', 'semana', 'quantidade',  # IDs, datetime e target
    'valor', 'num_transacoes',  # Features que vazam informação do futuro (se existirem)
    'mes', 'semana_ano'  # Features temporais brutas (mantemos sin/cos)
]

# Filtrar colunas que existem no DataFrame
exclude_features = [col for col in exclude_features if col in dados_treino_com_features.columns]

# Selecionar apenas colunas numéricas que não são datetime
numeric_columns = dados_treino_com_features.select_dtypes(include=[np.number]).columns
all_features = [col for col in numeric_columns if col not in exclude_features]

print(f'\n🎯 Preparando dados para treinamento:')
print(f'   • Target: {target}')
print(f'   • Features disponíveis: {len(all_features)}')
print(f'   • Features excluídas: {len(exclude_features)}')
print(f'   • Tipos de dados únicos: {dados_treino_com_features.dtypes.value_counts()}')

X_full = dados_treino_com_features[all_features]
y_full = dados_treino_com_features[target]

print(f'   • X_full shape: {X_full.shape}')
print(f'   • y_full shape: {y_full.shape}')
print(f'   • Tipos de dados em X_full: {X_full.dtypes.value_counts()}')

# Treinamento do modelo final
print('\n🚀 Treinando o modelo LightGBM final com todos os dados de 2022...')

lgb_params_final = {
    'objective': 'regression_l1',
    'metric': 'mae',
    'boosting_type': 'gbdt',
    'verbosity': -1,
    'random_state': 42,
    'n_jobs': -1
}

best_iteration = 200  # Usar a melhor iteração da validação anterior

train_full_lgb = lgb.Dataset(X_full, label=y_full, free_raw_data=False)  # free_raw_data=False para reutilização
final_model = lgb.train(lgb_params_final, train_full_lgb, num_boost_round=best_iteration)

print(f'✅ Modelo final treinado com sucesso em {best_iteration} iterações!')
print(f'   • Features utilizadas: {len(all_features)}')
print(f'   • Lista de features: {all_features[:10]}...')  # Mostrar primeiras 10

# Limpeza parcial de memória (manter dados necessários para teste)
print('\n🧹 Limpeza de memória...')
del train_full_lgb
gc.collect()

print('🎉 Pipeline de treinamento concluído com sucesso!')

📂 Carregando dados brutos de 2022...
   • Transações: (6560698, 11)
   • Produtos: (7092, 8)
   • PDVs: (14419, 4)

🔧 Aplicando engenharia de features...
🔧 Iniciando engenharia de features com Polars...
   • Transações: 6,560,698 registros
   • Produtos: 7,092 registros
   • PDVs: 14,419 registros
📝 Renomeando colunas para consistência...
✅ Colunas renomeadas com sucesso!
🔗 Fazendo joins dos dados...
📅 Processando datas...
   • Tipo atual da coluna semana: Date
🕒 Criando features temporais...
⏪ Criando features de lag...
📊 Criando features de rolling window...
🔢 Criando features de hash...
📈 Criando features históricas...
🔧 Preenchendo valores missing...
✅ Engenharia de features concluída!
   • Shape final: (6560698, 39)
   • Features criadas: 39

📊 Dados com features processados:
   • Shape: (6560698, 39)
   • Período: 2022-01-01 00:00:00 até 2022-12-01 00:00:00

📊 Otimização de memória e tratamento de missing values...
🔽 Aplicando downcasting...
📂 Otimizando categóricas...
🔧 Tratando

In [16]:
def engenharia_de_features(df_transacoes, df_produtos, df_pdvs):
    """
    Função para engenharia de features adaptada do notebook 02
    """
    import polars as pl
    
    print("🚀 Iniciando engenharia de features...")
    
    # 1. Renomear colunas para consistência com o pipeline original
    print("🔄 Renomeando colunas...")
    transacoes_pl = df_transacoes.rename({
        'internal_product_id': 'produto_id',
        'internal_store_id': 'pdv_id', 
        'reference_date': 'semana',
        'quantity': 'quantidade'
    })
    
    produtos_pl = df_produtos.rename({'produto': 'produto_id'})
    pdvs_pl = df_pdvs.rename({'pdv': 'pdv_id'})
    
    # 2. Joins para criar dataset base
    print("🔗 Realizando joins...")
    dados = transacoes_pl.join(produtos_pl, on="produto_id", how="left")
    dados = dados.join(pdvs_pl, on="pdv_id", how="left")
    
    # Filtrar apenas dados com vendas > 0 para otimização de memória
    dados = dados.filter(pl.col("quantidade") > 0)
    
    # 3. Conversão de data e ordenação
    print("📅 Processando datas...")
    # Verificar se a coluna já está em formato datetime
    if str(dados["semana"].dtype) != "Date":
        dados = dados.with_columns(pl.col("semana").str.to_datetime("%Y-%m-%d"))
    dados = dados.sort(["pdv_id", "produto_id", "semana"])
    
    # 4. Features temporais (mês, semana do ano, sin/cos para sazonalidade)
    print("📊 Criando features temporais...")
    dados = dados.with_columns([
        pl.col("semana").dt.month().alias("mes"),
        pl.col("semana").dt.week().alias("semana_do_ano"),
        (2 * 3.14159 * pl.col("semana").dt.month() / 12).sin().alias("mes_sin"),
        (2 * 3.14159 * pl.col("semana").dt.month() / 12).cos().alias("mes_cos"),
        (2 * 3.14159 * pl.col("semana").dt.week() / 52).sin().alias("semana_sin"),
        (2 * 3.14159 * pl.col("semana").dt.week() / 52).cos().alias("semana_cos")
    ])
    
    # 5. Features de lag (1, 2, 3, 4 semanas)
    print("⏮️ Criando features de lag...")
    dados = dados.with_columns([
        pl.col("quantidade").shift(1).over(["pdv_id", "produto_id"]).alias("lag_1"),
        pl.col("quantidade").shift(2).over(["pdv_id", "produto_id"]).alias("lag_2"),
        pl.col("quantidade").shift(3).over(["pdv_id", "produto_id"]).alias("lag_3"),
        pl.col("quantidade").shift(4).over(["pdv_id", "produto_id"]).alias("lag_4")
    ])
    
    # 6. Features de rolling window (médias móveis, std, min, max)
    print("📈 Criando features de rolling window...")
    dados = dados.with_columns([
        pl.col("quantidade").rolling_mean(window_size=4).over(["pdv_id", "produto_id"]).alias("media_4_semanas"),
        pl.col("quantidade").rolling_std(window_size=4).over(["pdv_id", "produto_id"]).alias("std_4_semanas"),
        pl.col("quantidade").rolling_min(window_size=4).over(["pdv_id", "produto_id"]).alias("min_4_semanas"),
        pl.col("quantidade").rolling_max(window_size=4).over(["pdv_id", "produto_id"]).alias("max_4_semanas")
    ])
    
    # 7. Features de hash para PDV e produto (redução de dimensionalidade)
    print("🏷️ Criando features de hash...")
    dados = dados.with_columns([
        pl.col("pdv_id").hash(seed=42) % 100,
        pl.col("produto_id").hash(seed=123) % 1000
    ])
    
    # 8. Features de interação (PDV x produto)
    print("🤝 Criando features de interação...")
    dados = dados.with_columns(
        (pl.col("pdv_id").cast(pl.Utf8) + "_" + pl.col("produto_id").cast(pl.Utf8)).alias("pdv_produto_combo")
    )
    dados = dados.with_columns(
        pl.col("pdv_produto_combo").hash(seed=456) % 10000
    )
    
    # 9. Limpeza de valores nulos (preencher com 0 para lags iniciais)
    print("🧹 Limpando valores nulos...")
    colunas_numericas = ["lag_1", "lag_2", "lag_3", "lag_4", "media_4_semanas", "std_4_semanas", "min_4_semanas", "max_4_semanas"]
    dados = dados.with_columns([
        pl.col(col).fill_null(0) for col in colunas_numericas
    ])
    
    # 10. Conversão para pandas para compatibilidade com sklearn
    print("🔄 Convertendo para pandas...")
    dados_pandas = dados.to_pandas()
    
    print(f"✅ Features criadas com sucesso! Shape: {dados_pandas.shape}")
    return dados_pandas